In [1]:
!pip install tsfresh

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, cv, Pool, CatBoostRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tsfresh import extract_features
from tqdm.auto import tqdm, trange
from sklearn.linear_model import RidgeClassifier, LogisticRegression, LogisticRegressionCV
import random

In [3]:
data = pd.read_csv('dataset.csv',parse_dates=['client_start_date','partnerrolestart_date'])
simple_sub = pd.read_csv('submission_example.csv')

In [4]:
def get_labels(data):
    date = pd.to_datetime('2020-9-01')
    labels = data.groupby('clientbankpartner_pin')[['client_start_date']].agg('max')
    labels['score'] = labels['client_start_date'].apply(lambda x: 0 if x >= date else 1)
    labels['label_count'] = data.groupby('clientbankpartner_pin')['client_start_date'].agg(lambda x: len([i for i in x.tolist() if i >= date]) )
    return labels

labels = get_labels(data)

In [5]:
train_data = data.copy()
train_data = train_data[train_data['client_start_date'] < pd.to_datetime('2020-9-01')]

In [6]:
def distance_calc(x):
    dist = []
    for i in range(len(x)-1):
        dist += [x[i+1] - x[i]]
    return dist

def create_time_features(df,report_date = '2020-12-01'):
    df['month'] = df['client_start_date'].dt.month
    df['day'] = df['client_start_date'].dt.day
    df['week'] = df['client_start_date'].dt.day.map(lambda x: x // 7)
    df['minute'] = df['client_start_date'].dt.minute
    df['year'] = df['client_start_date'].dt.year

    df['dayofweek'] = df['client_start_date'].dt.dayofweek
    df['all_year_time'] = df['client_start_date'].apply(lambda x: (x.year - 2019) * 365 + x.month * 30 + x.day)
    df['cl_is_weekend'] = (df['dayofweek'] > 4).astype('int16')
    
    df['time_to_report'] = pd.to_datetime(report_date) - df['client_start_date']
    df['days_to_report'] = df['time_to_report'].dt.days.astype(int)
    
    return df

def get_time_features(df):
    time_day_features = df.groupby('clientbankpartner_pin')['days_to_report'].agg(['mean', 'std', 'min', 'max', 'median','count'])
    time_day_features.columns = [f'time_day_features_{c}' for c in time_day_features.columns]
    time_day_features['time_day_features_diff'] = time_day_features['time_day_features_max'] - time_day_features['time_day_features_min']
    time_day_features['time_day_features_diversity'] = time_day_features['time_day_features_diff'] / time_day_features['time_day_features_count']
    
    diff_features = df.groupby('clientbankpartner_pin')['days_to_report'].agg(lambda x: distance_calc(x.tolist()))
    time_day_features['max_diff'] = diff_features.apply(lambda x: max(x) if x != []  else -100)
    time_day_features['min_diff'] = diff_features.apply(lambda x: min(x) if x != []  else -100)
    time_day_features['mean_diff'] = diff_features.apply(lambda x: np.mean(x) if x != []  else -100)
    time_day_features['std_diff'] = diff_features.apply(lambda x: np.std(x) if x != []  else -100)
    time_day_features['hard_time_step_polyfit'] = diff_features.map(lambda x: np.polyfit(range(len(x)), x, 1)[1] if len(x) > 2  else -100)
    time_day_features['hard_time_step_polyfit_st'] = diff_features.map(lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) > 2  else -100)

    return time_day_features

def get_src_code_features(df):
        src = df.groupby('clientbankpartner_pin')['partner_src_type_ccode'].agg(['first'])     
        return src


def partnerrolestart_date_feature(df,report_date = '2020-12-01'):
    df['days_from_start_to_report'] = pd.to_datetime(report_date) - df['partnerrolestart_date']
    start_day_features = df.groupby('clientbankpartner_pin')['days_from_start_to_report'].agg(['min'])
    start_day_features['days_from_start_to_report'] = start_day_features['min'].dt.days
    return start_day_features.drop(['min'],axis=1)

def get_user_features(df):
    user_strs = df.groupby('clientbankpartner_pin')['client_pin'].agg(lambda x:x.tolist())
    user_features = pd.DataFrame()
    user_features['user_nunique'] = user_strs.apply(lambda x: len(np.unique(x)))
    user_features['user_diversity'] = user_features['user_nunique'] / user_strs.apply(len)
    
    user_data = df.groupby('client_pin')['clientbankpartner_pin'].agg(['count'])
    user_data['client_pin'] = user_data.index
    user_data = user_data.rename({'count':'count_tr_per_user'},axis=1)
    user_data.index = range(len(user_data))
    df = df.merge(user_data)
    
    user_features['user_max_act'] =  df.groupby('clientbankpartner_pin')['count_tr_per_user'].agg('max')
    user_features['user_mean_act'] =  df.groupby('clientbankpartner_pin')['count_tr_per_user'].agg('mean')
    return user_features


def last_count(df,bins=[5,10,15,20,25,30,35,45,50,60]):
    feature_df = pd.DataFrame(index=df.groupby('clientbankpartner_pin')['client_pin'].agg('count').index)
    for x in bins:
        feature_df[f'count_{x}_prev'] = df[df['days_to_report'] > x].groupby('clientbankpartner_pin')['client_pin'].agg('count')
        feature_df[f'count_{x}_last'] = df[df['days_to_report'] <= x].groupby('clientbankpartner_pin')['client_pin'].agg('count')
        feature_df[f'percent_{x}'] =  feature_df[f'count_{x}_last'] / (feature_df[f'count_{x}_last'] + feature_df[f'count_{x}_prev'])
        
    return feature_df.fillna(-1)

def create_tsfresh_features(df):
    df_features = extract_features(train_data,column_id='clientbankpartner_pin',column_value='days_to_report')
    return df_features

def compute_all_features(data,report_date):
    data = create_time_features(data,report_date)
    time_features = get_time_features(data)
    src_features = get_src_code_features(data)
    start_day_features = partnerrolestart_date_feature(data,report_date)
    user_features = get_user_features(data)
    last_count_feats = last_count(data)
    
    all_data = pd.concat([time_features,
                      src_features,
                      start_day_features,
                      last_count_feats,
                      user_features,
                      ],axis=1)
    return all_data

In [7]:
train_feats = compute_all_features(train_data,'2020-9-01')

In [8]:
#train_tsmix_feats = extract_features(train_data,column_id='clientbankpartner_pin',column_value='days_to_report')
#train_tsmix_feats.to_parquet('train_tsmixer.parquet')
train_tsmix_feats = pd.read_parquet('train_tsmixer.parquet')

In [9]:
train_embeds = pd.read_parquet('train_embeds.parquet')

In [10]:
all_train_data = pd.concat([train_feats.sort_index(),
                            train_tsmix_feats.sort_index(),
                            train_embeds.sort_index(),
                            labels.loc[train_feats.index]],axis=1)

In [11]:
all_train_data = all_train_data.T.drop_duplicates().T

In [12]:
def get_val_splits(data,num_splits=10,n_val=1000,random_seed=56):
    date = pd.to_datetime('2020-6-01')
    labels = data.groupby('clientbankpartner_pin')[['client_start_date']].agg('max')
    good_partners = labels[labels['client_start_date'] >= date].index.tolist()
    all_partners = labels.index.tolist()
    train_idxs, val_idxs = [],[]
    for i in range(num_splits):
        random.seed(random_seed+i)
        random.shuffle(good_partners)
        val_part = good_partners[:n_val]
        tr_part = [part for part in all_partners if part not in val_part]
        train_idxs.append(tr_part)
        val_idxs.append(val_part)
    return train_idxs, val_idxs

train_splits, eval_splits = get_val_splits(train_data)

In [13]:
params = {'iterations':1_000,
          'loss_function':'CrossEntropy',
          'learning_rate':0.01,
          'task_type':'CPU',
          'min_child_samples':20,
          'max_depth':5,
          'eval_metric':'AUC',
          'random_seed':56}

drop_cols = ['label_count','score','client_start_date']

In [14]:
models = []
scores = []
for i in trange(len(train_splits)):
    train_pool = Pool(data=all_train_data.loc[train_splits[i]].drop(drop_cols,axis=1),
                      label=all_train_data.loc[train_splits[i]]['score'],
                      cat_features=['first'])
    eval_pool = Pool(data=all_train_data.loc[eval_splits[i]].drop(drop_cols,axis=1),
                      label=all_train_data.loc[eval_splits[i]]['score'],
                      cat_features=['first'])
    
    cbm = CatBoostClassifier(**params)
    cbm.fit(train_pool,eval_set=eval_pool,verbose=300)
    scores.append(cbm.get_best_score())
    models.append(cbm)

  0%|          | 0/10 [00:00<?, ?it/s]

0:	test: 0.6999417	best: 0.6999417 (0)	total: 153ms	remaining: 2m 32s
300:	test: 0.7601790	best: 0.7602599 (297)	total: 21.7s	remaining: 50.3s
600:	test: 0.7617479	best: 0.7618799 (490)	total: 42.2s	remaining: 28s
900:	test: 0.7635106	best: 0.7635787 (894)	total: 1m 2s	remaining: 6.86s
999:	test: 0.7639023	best: 0.7639534 (989)	total: 1m 8s	remaining: 0us

bestTest = 0.7639534042
bestIteration = 989

Shrink model to first 990 iterations.
0:	test: 0.7437000	best: 0.7437000 (0)	total: 92.4ms	remaining: 1m 32s
300:	test: 0.7960504	best: 0.7960504 (300)	total: 21.6s	remaining: 50.2s
600:	test: 0.7987065	best: 0.7988522 (577)	total: 41.7s	remaining: 27.7s
900:	test: 0.7987169	best: 0.7991977 (636)	total: 1m 2s	remaining: 6.82s
999:	test: 0.7986961	best: 0.7991977 (636)	total: 1m 8s	remaining: 0us

bestTest = 0.7991977485
bestIteration = 636

Shrink model to first 637 iterations.
0:	test: 0.7033006	best: 0.7033006 (0)	total: 94.3ms	remaining: 1m 34s
300:	test: 0.7735355	best: 0.7735355 (300)

In [19]:
np.mean([x['validation']['AUC'] for x in scores])

0.7796459349156033

In [20]:
imp = models[0].get_feature_importance(prettified=True).set_index('Feature Id')
for i in range(1,10):
    imp += models[i].get_feature_importance(prettified=True).set_index('Feature Id')

In [21]:
imp['is_emb'] = list(map(lambda x: 'emb' in x, imp.index.tolist()))
imp = imp[~imp['is_emb']]

In [28]:
best_features = imp.sort_values(by='Importances')[::-1][:128].index.tolist() + [f'emb_000{i}' for i in range(10)] + [f'emb_00{i}' for i in range(10,32)]

In [29]:
imp.sort_values(by='Importances')[::-1].head(30)

Importances  is_emb
Feature Id                                                             
time_day_features_min                                 87.589535   False
count_60_last                                         64.380334   False
count_45_last                                         35.611658   False
count_50_last                                         25.205623   False
count_35_last                                         23.324624   False
days_to_report__quantile__q_0.1                       18.149922   False
count_25_last                                         15.983998   False
count_30_last                                         12.210550   False
days_to_report__quantile__q_0.2                       10.165276   False
days_to_report__quantile__q_0.3                        8.506198   False
count_20_last                                          8.440073   False
days_to_report__variation_coefficient                  8.000002   False
days_to_report__quantile__q_0.4                        6.752258   False
count_15_last                                          5.625748   False
time_day_features_mean                                 4.744816   False
time_day_features_median                               4.643156   False
days_to_report__index_mass_quantile__q_0.8             3.947675   False
hard_time_step_polyfit_st                              3.731395   False
days_to_report__root_mean_square                       3.411802   False
percent_15                                             3.089943   False
first                                                  2.815781   False
days_to_report__benford_correlation                    2.705587   False
days_to_report__quantile__q_0.6                        2.672348   False
percent_30                                             2.640493   False
days_to_report__change_quantiles__f_agg_"mean"_...     2.603877   False
percent_25                                             2.568059   False
days_to_report__change_quantiles__f_agg_"var"__...     2.470383   False
days_to_report__quantile__q_0.7                        2.464888   False
time_day_features_diversity                            2.432268   False
days_to_report__change_quantiles__f_agg_"mean"_...     2.366294   False

In [30]:
params = {'iterations':2_000,
          'loss_function':'CrossEntropy',
          'learning_rate':0.01,
          'task_type':'CPU',
          'min_child_samples':20,
          'max_depth':4,
          'eval_metric':'AUC',
          'random_seed':56}

In [31]:
models = []
scores = []
for i in trange(len(train_splits)):
    train_pool = Pool(data=all_train_data.loc[train_splits[i]][best_features],
                      label=all_train_data.loc[train_splits[i]]['score'],
                      cat_features=['first'])
    eval_pool = Pool(data=all_train_data.loc[eval_splits[i]][best_features],
                      label=all_train_data.loc[eval_splits[i]]['score'],
                      cat_features=['first'])
    
    cbm = CatBoostClassifier(**params)
    cbm.fit(train_pool,eval_set=eval_pool,verbose=300)
    scores.append(cbm.get_best_score())
    models.append(cbm) 

  0%|          | 0/10 [00:00<?, ?it/s]

0:	test: 0.7216387	best: 0.7216387 (0)	total: 19.3ms	remaining: 38.5s
300:	test: 0.7596830	best: 0.7596830 (300)	total: 5.25s	remaining: 29.7s
600:	test: 0.7624909	best: 0.7625548 (598)	total: 10s	remaining: 23.3s
900:	test: 0.7647836	best: 0.7647964 (897)	total: 14.8s	remaining: 18s
1200:	test: 0.7655245	best: 0.7657331 (1060)	total: 19.6s	remaining: 13s
1500:	test: 0.7667890	best: 0.7667933 (1499)	total: 24.5s	remaining: 8.14s
1800:	test: 0.7670700	best: 0.7670700 (1800)	total: 29.2s	remaining: 3.22s
1999:	test: 0.7669551	best: 0.7671679 (1803)	total: 32.3s	remaining: 0us

bestTest = 0.7671679347
bestIteration = 1803

Shrink model to first 1804 iterations.
0:	test: 0.7478674	best: 0.7478674 (0)	total: 15.9ms	remaining: 31.8s
300:	test: 0.7975741	best: 0.7975741 (300)	total: 4.9s	remaining: 27.6s
600:	test: 0.8005903	best: 0.8006840 (596)	total: 9.66s	remaining: 22.5s
900:	test: 0.8019476	best: 0.8021182 (885)	total: 14.4s	remaining: 17.5s
1200:	test: 0.8014834	best: 0.8023139 (932)	t

In [ ]:
np.mean([x['validation']['AUC'] for x in scores])

In [ ]:
test_data = data.copy()
test_feats = compute_all_features(test_data,'2020-12-01')

In [ ]:
#test_tsmix_feats = extract_features(test_data,column_id='clientbankpartner_pin',column_value='days_to_report')
#test_tsmix_feats.to_parquet('test_tsmixer.parquet')
test_tsmix_feats = pd.read_parquet('test_tsmixer.parquet')

In [ ]:
test_embeds = pd.read_parquet("test_embeds.parquet")

In [ ]:
all_test_data = pd.concat([test_feats.sort_index(),test_tsmix_feats.sort_index(),test_embeds.sort_index()],axis=1)

In [ ]:
simple_sub['clientbankpartner_pin']

In [ ]:
all_test_data_lc = all_test_data.loc[simple_sub['clientbankpartner_pin']]

In [ ]:
test_pool = Pool(all_test_data_lc[best_features],
                 cat_features=['first'])

In [ ]:
scores = np.mean([cbm.predict_proba(test_pool)[:,1] for cbm in models],axis=0)

In [ ]:
simple_sub['score'] = scores

In [32]:
simple_sub.to_csv('CatBoost_Embeds.csv',index=False)